<a href="https://colab.research.google.com/github/parthag1201/DocReversalEngine/blob/main/FS_TS_Crew_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages for crew and langchain
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_google_genai fpdf2 markdown2

In [2]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [3]:
# pdf read and write
from fpdf import FPDF
from markdown2 import Markdown

In [4]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Import crewAI library
from crewai import Agent, Task, Crew, Process

In [6]:
import os
from google.colab import userdata

In [ ]:
!pip install PyPDF2

In [11]:
# --- File and Folder Handling ---

# 1. Read code from the 'code_files' folder
import PyPDF2 # Import PyPDF2 here
code_folder_path = 'code_files'
if not os.path.exists(code_folder_path):
    os.makedirs(code_folder_path)
    # Create a dummy file for demonstration if the folder is empty
    with open(os.path.join(code_folder_path, 'sample_code.txt'), 'w') as f:
        f.write("REPORT Z_SAMPLE_REPORT.") # Add some dummy code

code_input = ""
for filename in os.listdir(code_folder_path):
    if filename.endswith(".txt"):
        with open(os.path.join(code_folder_path, filename), 'r') as f:
            code_input += f.read() + "\n\n"

# 2. Read the template from an uploaded PDF
template_pdf_path = 'template.pdf'
template_text = ""
try:
    with open(template_pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            template_text += page.extract_text()
except FileNotFoundError:
    print("Please upload a 'template.pdf' file to use as the template.")
    # Create a dummy template.pdf for demonstration if it doesn't exist
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="This is a dummy template.", ln=True, align='C')
    pdf.output(template_pdf_path)
    print("A dummy 'template.pdf' has been created for you.")
    with open(template_pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            template_text += page.extract_text()

In [21]:
# Agents
from langchain_community.llms import HuggingFaceHub # Import necessary class if using Hugging Face models
from langchain_google_genai import ChatGoogleGenerativeAI # Import the correct class for Gemini

# Initialize the Gemini LLM
os.environ['GOOGLE_API_KEY']=userdata.get('gemini_pro')
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")


abap_code_analyst = Agent(
    role="Senior ABAP Code Analyst",
    goal="Dissect the provided ABAP report code to extract all technical details and the logic flow of the report."
    "Focus on identifying data sources, selection screens, core logic, and output structures."
    "You need to go through each and every section of report to the extent form where a good Functional and Technical specification can be derived out of it"
    "This is the ABAP report code : '{code_files}'",
    backstory="""You are an expert ABAP developer with decades of experience. You have an exceptional eye for detail
    and can instantly understand the flow and structure of any ABAP program. Your task is to analyze the
    code and provide a structured, raw analysis for the documentation team.""",
    allow_delegation=False,
    verbose=True,
    llm=gemini_llm,
)

functional_spec_drafter = Agent(
    role="Business Systems Analyst",
    goal="Translate the technical analysis of an ABAP report into a clear and concise"
   "functional specification document for business stakeholders."
   "Refer to this professional template for creating a business functional specification document. '{template_text}'",
    backstory="""You are a bridge between the technical team and the business. You excel at taking complex
    technical jargon and reframing it in terms of business processes and objectives. Your functional
    specifications are legendary for their clarity and business relevance, enabling stakeholders to
    understand exactly what a report does.""",
    allow_delegation=True,
    verbose=True,
    llm=gemini_llm,
)

technical_spec_writer = Agent(
    role="Technical Documentation Specialist",
    goal="""Create a comprehensive and detailed technical specification document based on the
    ABAP code analysis."""
    "Refer to this professional template for creating a business technical specification document. '{template_text}'",
    backstory="""You are a meticulous technical writer who specializes in creating documentation for developers.
    Your specifications are precise, well-structured, and follow industry best practices. Your work ensures
    that any developer can pick up the report for maintenance or enhancement with complete clarity
    on its internal workings.""",
    allow_delegation=True,
    verbose=True,
    llm=gemini_llm,
)

manager_agent = Agent(
    role='Manager Agent',
    goal='Oversee the generation and refinement of Functional and Technical Specifications, facilitate collaboration, incorporate reviewer feedback, and ensure final output meets all requirements.'
    'The FS and TS reports must contain all the important information extracted from the report fror the business team to understand each and every ascpet of the report easily',
    backstory="""You are an experienced SAP S4 project manager with a deep understanding of software development life cycles and well versed with the good practises of SAP, Busness needs and ABAP Development.
    You excel at breaking down complex tasks, coordinating teams, incorporating feedback, and ensuring quality deliverables.
    You are adept at communicating with both technical and business stakeholders.""",
    verbose=True,
    allow_delegation=True,
    llm=gemini_llm,
)

output_reviewer = Agent(
    role='Output Reviewer (Business Knowledge)',
    goal='Review consolidated Functional and Technical Specifications and provide constructive feedback to the Manager Agent for refinement, focusing on business alignment and completeness.',
    backstory="""You are a business stakeholder with a keen eye for detail and a deep understanding of the organizational goals and user needs.
    You critically evaluate specifications, ensuring they meet intended business objectives and provide actionable feedback for improvement.""",
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm,
)

In [22]:
# Tasks

analyze_code_task = Task(
    description="""Analyze the following ABAP report code: '{code_files}'.
                   Your analysis must cover these key areas:
                   1.  **Data Sources:** Identify all database tables, views, or structures being used.
                   2.  **Selection Screen:** Detail all SELECT-OPTIONS and PARAMETERS, including their technical IDs and associated data elements.
                   3.  **Core Processing Logic:** Summarize the main logic blocks, including loops, conditionals (IF/CASE statements), and subroutine calls (PERFORM).
                   4.  **Data Output:** Describe how the final data is presented (e.g., ALV grid, classical report WRITE statements) and list the fields being displayed.""",
    expected_output="""A structured text document containing a raw, point-by-point technical breakdown of the ABAP report.
                       This output should be purely technical and serve as the foundational information for other agents.""",
    agent=abap_code_analyst,
)

# Task for the Functional Spec Drafter
functional_spec_task = Task(
    description="""Using the technical analysis provided by the ABAP Code Analyst, create a functional specification document.
                   The document should explain the report's purpose and functionality from a business user's perspective.
                   It must include:
                   1.  **Report Purpose:** A high-level summary of what the report achieves.
                   2.  **Selection Criteria:** Explain the input fields in simple business terms (e.g., "User can filter by Sales Document Number").
                   3.  **Processing Logic Summary:** Describe what the report does with the data in plain language (e.g., "The report calculates the total net value for the selected sales orders").
                   4.  **Output Description:** Describe the layout and columns of the final report from a user's point of view.""",
    expected_output="""A well-formatted functional specification document in Markdown. The language should be clear,
                       non-technical, and focused on the business value and utility of the report.""",
    agent=functional_spec_drafter,
)

# Task for the Technical Spec Writer
technical_spec_task = Task(
    description="""Using the detailed technical analysis from the ABAP Code Analyst, create a formal technical specification document.
                   This document must be structured for a developer audience and include:
                   1.  **Program Details:** Program ID, Title, and a brief technical overview.
                   2.  **Data Declarations:** List of key tables, internal tables, and complex variables.
                   3.  **Selection Screen Objects:** A table listing each selection screen field with its technical name and properties.
                   4.  **Detailed Logic Flow:** A step-by-step description of the program's execution logic, referencing specific subroutines and key logic blocks.
                   5.  **Output Details:** A technical breakdown of the output display, including the field catalog for ALV grids or the format of WRITE statements.""",
    expected_output="""A comprehensive technical specification document in Markdown. It should be precise, detailed, and
                       formatted professionally to serve as official technical documentation.""",
    agent=technical_spec_writer,
)


# Manager Task for initial consolidation
initial_consolidation_task = Task(
    description="""Consolidate the initial Functional and Technical Specifications into a single, cohesive document.
    Ensure both specifications are present and well-formatted for review.
    """,
    expected_output="A single Markdown document containing the initial Functional Specification and Technical Specification, clearly separated.",
    agent=manager_agent,
    context=[functional_spec_task, technical_spec_task],
    llm=gemini_llm,
)

# Output Reviewer Task to provide feedback to the manager
output_review_feedback_task = Task(
    description="""Review the provided consolidated Functional and Technical Specifications.
    Focus on business alignment, clarity, completeness, and accuracy from a business perspective.
    Provide constructive feedback to the Manager Agent, detailing any required changes or improvements.
    The output should be a clear, concise, and structured list of actionable feedback points, not a final report.
    """,
    expected_output="A structured list of feedback points (e.g., bullet points, numbered list) for the Manager Agent to use for refining the specifications. Example: '- Section X needs more detail on Y. - Clarify business impact of Z. - Ensure consistency between FS and TS for ABC.'",
    agent=output_reviewer,
    context=[initial_consolidation_task], # Reviewer reviews the initial consolidated output
    llm=gemini_llm,
)

# Manager Task for final refinement based on feedback
final_specification_task = Task(
    description="""Based on the initial Functional and Technical Specifications and the feedback provided by the Output Reviewer,
    make necessary revisions to both specifications. Your goal is to produce the final, polished, and
    approved Functional and Technical Specifications in a single consolidated document.
    The output must contain the actual Functional Specification and Technical Specification,
    incorporating all valid feedback. Ensure clear separation between the FS and TS sections in the final output.
    """,
    expected_output="A single, comprehensive Markdown document containing the final, refined Functional Specification and Technical Specification, clearly separated (e.g., using distinct headings like '# Functional Specification' and '# Technical Specification') and incorporating all reviewer feedback.",
    agent=manager_agent,
    context=[initial_consolidation_task, output_review_feedback_task], # Manager uses both initial specs and feedback
    llm=gemini_llm,
    output_file='final_specifications.md' # Final output will be saved here
)

In [23]:
# Call the crew

abap_doc_crew = Crew(
    agents=[
        abap_code_analyst,
        functional_spec_drafter,
        technical_spec_writer,
        manager_agent,
        output_reviewer
    ],
    tasks=[
        analyze_code_task,
        functional_spec_task,
        technical_spec_task,
        initial_consolidation_task,
        output_review_feedback_task,
        final_specification_task # This task produces the final output after review
    ],
    process=Process.sequential,
    verbose=2
)

In [ ]:
# --- Execution ---
if code_input and template_text:
    result = abap_doc_crew.kickoff(
        inputs={
            'code_files': code_input,
            'template_text': template_text
        }
    )

In [34]:
from IPython.display import Markdown
Markdown(result)

### **Interface Functional & Technical Specification**

**SR162489 - Automatic Calculation of QM Inspection Lot End Date**

---

<br>

**SAP Functional & Technical Specification Document - Interfaces**

| | |
| :--- | :--- |
| **Object ID** | SR162489 |
| **Object Type** | Enhancement / Business Logic |
| **Object Title** | Automatic Calculation of QM Inspection Lot End Date |
| **Process Area** | **X** MTD (Make to Deliver) |
| **Sub Process Area** | Quality Management (QM) |
| **Complexity** | **X** Medium |
| **Interface Type** | **X** Inbound (Internal system process) |
| **Source -> Target** | SAP S/4HANA -> SAP S/4HANA |
| **Rollout information** | < R1 > |

<br>
<br>

### **Table of Contents**

1.  Document Control Information
2.  Executive Summary
3.  Business Process Considerations
4.  Functional Design Considerations
5.  Nonfunctional requirements
6.  Technical Interface Specifications
7.  Unit Test Scenarios
8.  Attachments and Documentation

<br>
<br>

### **1. Document Control Information**
*(This section would be filled in during the project lifecycle.)*

| Date Opened | Description | Assigned to | Due Date | Status |
| :--- | :--- | :--- | :--- | :--- |
| *[Date]* | **[Open Item #1]** Business to confirm if any specific Return scenarios (e.g., by material type or return reason) require a fixed inspection duration instead of the default 'indefinite' date. | Business Lead - Quality Mgmt | *[Date + 2 weeks]* | Open |

---

### **2. Executive Summary**

**2.1 Overview**
This document describes an automated enhancement to the standard Quality Management (QM) process. This enhancement automatically calculates and sets a more appropriate "Inspection End Date" for specific Quality Inspection Lots at the time of their creation. The calculation logic depends on the business context, such as processing a customer return, receiving a material with a mandatory "cool-down" period, or handling an internal stock transfer.

**2.2 Business Driver**
The standard SAP system often assigns a generic inspection end date that does not meet specific business requirements. This creates several challenges:
*   **Process Inefficiency:** Quality Planners must manually review and update inspection end dates for numerous lots, which is time-consuming and prone to error.
*   **Compliance Risk:** Materials requiring a "cool-down" period (a duration where the material must settle before use) could be released for use prematurely if the inspection lot expires based on a standard, shorter timeframe.
*   **Data Inaccuracy:** Inspection lots for returns may expire unnecessarily, causing confusion and requiring manual intervention to keep them active for analysis.

This enhancement addresses these issues by automating the date calculation, ensuring process compliance, improving data accuracy, and reducing manual workload.

**2.3 As-Is process**
Currently, when an inspection lot is created, the system assigns a standard end date based on basic configuration. For special cases like returns or materials with cool-down requirements, a Quality Planner must manually identify these lots and change the inspection end date using transaction QA02 (Change Inspection Lot).

**2.4 To-be process**
With this enhancement, the system will automatically identify predefined business scenarios during goods movement transactions (e.g., MIGO). Based on the scenario, it will apply a specific business rule to calculate and set the correct inspection end date on the inspection lot as it is being created. This process is seamless and requires no manual intervention.

**2.5 Business Risks/Issues**
*Failure of the automated logic poses significant business risks that could impact product quality, regulatory compliance, and inventory management. The specific risks associated with each business rule are:*

*   **Rule 1 (Returns Processing):** If the logic fails to set an 'indefinite' inspection end date for a return, the system may default to a standard, shorter duration. This could lead to the premature, automated write-off or scrapping of perfectly good returned inventory, resulting in direct financial loss and inaccurate stock levels.
*   **Rule 2 ('Cool-Down' Period):** A failure to correctly apply the mandatory cool-down period for specific materials is the most critical risk. It could lead to the premature quality release of materials that have not yet stabilized (e.g., curing time for chemicals, temperature acclimation for sensitive goods). This introduces a severe risk of distributing defective or unsafe products to customers, potentially causing harm, triggering product recalls, damaging brand reputation, and incurring regulatory penalties.
*   **Rule 3 (History-Based Dating):** If the system cannot find a valid historical lot to copy the end date from, the new lot may receive an incorrect default date. An unnecessarily long date ties up valuable inventory, negatively impacting working capital. An unnecessarily short date may not provide sufficient time for proper testing, leading to rushed quality decisions or the unnecessary expiry of stock in transit.

**2.6 Assumptions**
*   All required master data (e.g., Material Master QM view) will be correctly maintained prior to go-live.
*   The logic is being implemented in S/4HANA and will trigger upon the creation of a Quality Management inspection lot.
*   **Data Governance:** The custom table `ZGMTD_CLD_PGI`, which stores the material- and plant-specific cool-down periods, is a critical configuration table.
    *   **Business Data Owner:** The **Master Data Governance Team** is the official owner of this table's content.
    *   **Governance Process:** All requests for adding, changing, or deleting entries in this table must be initiated by the **Plant Quality Manager** or a delegate. The request must be formally submitted via a service management ticket, which will then be routed to the Master Data Governance Team for implementation after verification. This ensures proper controls and audit trails for changes.

**2.7 Impacted Systems**
*   **X** S/4 HANA

<br>

### **3. Business Process Considerations**

**3.1 Process Description and Flow**
This enhancement is not a user-executed report but a background process that triggers automatically during a goods movement that creates a QM Inspection Lot.

The process flow is as follows:
1.  A user performs a goods movement transaction (e.g., Goods Receipt, Stock Transfer, Return).
2.  The standard SAP process begins to create an inspection lot.
3.  Before the lot is saved, this enhancement is triggered via a BAdI.
4.  The enhancement checks if the material, movement type, and/or inspection type match a predefined business rule.
5.  If a rule is matched, the enhancement calculates a new "Inspection End Date" and updates the inspection lot data.
6.  The standard SAP process continues and saves the inspection lot with the new, correct end date.

**3.2 Data Source**
*   **Primary Source:** Real-time inspection lot data being generated by a standard SAP transaction (e.g., MIGO).
*   **Supporting Data:**
    *   Material Master Data (QM Inspection Type view).
    *   Plant Data (Factory Calendar).
    *   Custom Configuration Table `ZGMTD_CLD_PGI` (stores cool-down times per Material and Plant).
    *   Historical Inspection Lot Data (Table QALS).

**3.3 Trigger**
The process is triggered automatically by the system during the creation of a Quality Inspection Lot under specific conditions. It is not manually started by a user.

**3.4 Processing Options**
*   **Processing Mode:** **X** Real Time
*   **Processing Type:** **X** Asynchronous (from the user's perspective, it's part of the single transaction they are performing).
*   **Frequency:** **X** On Demand (triggers whenever a relevant goods movement occurs).

**3.5 Dependencies**
The successful execution of this enhancement depends on the correct configuration of Material Master data and the custom "Cool-Down" table `ZGMTD_CLD_PGI`.

<br>

### **4. Functional Design Considerations**

**4.1 Interface Details**

**4.1.2 Mapping and Transformation**
*The core logic is executed via a custom enhancement (BAdI) triggered during the creation of an inspection lot (transaction QA01 or automatic creation via goods movement). The system will determine the inspection lot end date based on the following prioritized business rules:*

**Rule 1: Returns Inspection (Highest Priority)**
*   **Logic:** If the inspection lot is created for a return (Movement Type '655') or is of Inspection Type '05'.
*   **Action:** The system sets the **Inspection End Date** to the last possible date of the latest year defined in the plant's factory calendar (e.g., 12/31/2999).
*   **Business Rationale:** Returns require detailed manual investigation to determine their disposition (return to stock, refurbish, scrap). This process has no fixed timeline, so an indefinite date prevents the lot from appearing on exception reports for being overdue.
*   **Validation Note:** This logic applies to all return movement types by default. If specific returns require a fixed inspection duration, this will be handled as a future enhancement.

**Rule 2: Material 'Cool-Down' Period (Second Priority)**
*   **Logic:** If the inspection lot is for a goods receipt (Movement Type '101') of a material requiring a post-receipt stabilization or "cool-down" period (Inspection Type '08' or 'Z01').
    *   The system will look up the **Material Number** and **Plant** in the custom table `ZGMTD_CLD_PGI`.
    *   If a unique, valid entry is found, the 'Inspection End Date' will be calculated as: `Inspection Lot Creation Date + Cool-Down Days`.
    *   **Data Integrity Check:** If the lookup returns **multiple** active entries for the same Material/Plant combination, this indicates a master data error. The automatic calculation will be skipped, and the lot will be flagged in the daily **Exception Monitoring Report** for manual review and correction by the Master Data Governance Team.
*   **Business Rationale:** Certain materials require a fixed period of time after receipt before they can be safely tested or used. This rule automates the enforcement of that mandatory waiting period, ensuring product quality and safety. The lookup keys are Material and Plant to ensure the rule applies to the material regardless of the vendor or inbound delivery.

**Rule 3: History-Based Date for Stock Transfers (Third Priority)**
*   **Logic:** If the inspection lot is created for an internal stock transfer (e.g., Movement Types '322', '349', or 'Z49' with Inspection Types '08' or 'Z01').
    *   The system will search for a pre-existing lot for the same **Material** and **Batch**. The search is prioritized:
        1.  First, it searches for a previous inspection lot with Inspection Type **'04' (Goods Receipt from Production)**.
        2.  If no '04' lot is found, it then searches for a previous lot with Inspection Type **'08' or 'Z01'**.
    *   The 'Inspection End Date' from the most recently created lot found through this search will be compared to today's date. The new Inspection End Date will be set to whichever date is later.
*   **Business Rationale:** This ensures that the original quality expiration date established when the batch was produced is consistently maintained as it moves between plants. It prevents a batch from having its quality timeline reset during internal transfers.

**Default Behavior:**
*   If none of the rules above apply, the system will use the standard SAP logic to calculate the inspection end date.

**4.2 Error Handling, Reconciliation, and Reporting**
*To ensure system integrity and provide visibility into failures of the custom logic, a robust error handling and reporting process is required.*

**Error Logging Process:**
*Any failure within the enhancement (e.g., data inconsistency, unexpected system error) will be logged. The enhancement will gracefully exit, allowing the standard SAP process to complete. The specific failure reason will be captured for reporting.*

**Process for Error Resolution:**
*The responsible business teams (Quality Management, Master Data Governance) will use the Exception Monitoring Report to identify, investigate, and resolve issues. Resolution typically involves correcting the source data (e.g., in table `ZGMTD_CLD_PGI` or Material Master) and manually updating the inspection lot end date via transaction QA02.*

**4.2.2 Exception Monitoring Report (New Requirement)**
*A new report is required to provide business users with visibility into all instances where the automatic end-date calculation logic could not be successfully executed.*

*   **Report Title:** ZQM_ENDDATE_FAILURE - QM Lot End Date Logic Exceptions
*   **Purpose:** To identify inspection lots that require manual review and correction of the inspection end date due to a failure in the automated logic. This report is the primary control mechanism for mitigating the risks of "silent failures."
*   **Selection Criteria:** Plant, Material, Inspection Lot Creation Date Range.
*   **Report Output Fields:**
    *   Inspection Lot Number
    *   Material Number
    *   Material Description
    *   Batch Number
    *   Plant
    *   Inspection Type
    *   Movement Type
    *   Inspection Lot Creation Date & Time
    *   **Reason for Failure:** A clear text field explaining why the logic failed (e.g., "Multiple cool-down records found," "No historical lot found for stock transfer," "Invalid data in ZGMTD_CLD_PGI").
*   **Access & Security:** The report shall be accessible to users with roles assigned to the **Plant Quality Management** and **Master Data Governance** teams.
*   **Execution & Scheduling:** The report must be available to run on-demand. Additionally, it will be scheduled to run daily (e.g., at 6:00 AM) and the output emailed to a distribution list of key quality and master data personnel for proactive review.

<br>

### **5. Nonfunctional requirements**
*(This section would be filled in during the project lifecycle.)*

---

### **6. Technical Interface Specifications**

#### **6.1. Enhancement/Object Details**

| Item | Description |
| :--- | :--- |
| **Object Type** | Business Add-In (BAdI) Implementation |
| **Enhancement Spot** | `QPL10001` - Enhancements for Inspection Lot Creation |
| **BAdI Definition** | `QPL1_INSP_LOT_CREATE` - BAdI for Inspection Lot Creation |
| **BAdI Implementation Name** | `ZMTD_QPL1_INSP_LOT_CREATE` |
| **Implementing Class** | `ZCL_MTD_QPL1_INSP_LOT_CREATE` |
| **Method to Implement** | `IF_EX_QPL1_INSP_LOT_CREATE~CREATE_INSP_LOT` |
| **Legacy Object to Decommission**| User-Exit `EXIT_SAPLQPL1_002` (Code within include `ZXQPLU02` should be migrated to the BAdI and the exit implementation deactivated/cleared). |

#### **6.2. RICEFW Object Summary**

| ID | Type | Object Name | Description |
| :--- | :--- | :--- | :--- |
| 1 | Enhancement | `ZMTD_QPL1_INSP_LOT_CREATE` | BAdI implementation to modify inspection lot end date based on business rules. |
| 2 | Class | `ZCL_MTD_QPL1_INSP_LOT_CREATE` | Class containing the core logic for the BAdI implementation. |
| 3 | Table | `ZGMTD_CLD_PGI` | Custom table storing Cool-Down duration per Material and Plant. |
| 4 | Class | `ZCL_GXAP_COMMON_CLASS` | Common framework class to check if the enhancement is active. |
| 5 | Report | `ZQM_ENDDATE_FAILURE` | **(New)** Daily report to identify inspection lots where the logic failed due to configuration issues (e.g., multiple cool-down records). |


#### **6.3. Detailed Logic Flow**

The logic will be implemented within the method `IF_EX_QPL1_INSP_LOT_CREATE~CREATE_INSP_LOT` of the class `ZCL_MTD_QPL1_INSP_LOT_CREATE`. The changing parameter `C_QALS` will be used to update the inspection lot data. The BAdI provides importing parameters like `I_QALS` (Inspection Lot Header) and `I_IMSEG` (Material Document Item).

**Execution Trigger:** The BAdI is triggered during the creation of an inspection lot, for example, via goods movements in transactions like MIGO, MB1B, etc.

**Step 1: Central Activation Check**
*   The first step within the method is to call the central activation check to ensure the enhancement `SR162489` is globally active for the 'MTD' business area. If not active, the method will exit immediately.

**Step 2: Rule 1 - Return Orders**
*   **Condition:** If `c_qals-art = '05'` OR `i_imseg-bwart = '655'`.
*   **Logic:**
    1.  Determine the highest year available in the factory calendar for the plant (`c_qals-werk`).
    2.  Update `c_qals-paendterm` with the last day of that year (e.g., '29991231').
    3.  Set `e_active = 'X'` and exit further processing.

**Step 3: Rule 2 - Cool-Down Period Calculation**
*   **Condition:** If `c_qals-art` is `'08'` or `'Z01'` AND `i_imseg-bwart = '101'`.
*   **Logic:**
    1.  `SELECT * FROM zgmtd_cld_pgi` into an internal table where `matnr = c_qals-matnr` and `werks = c_qals-werk`.
    2.  Check the number of records found (`sy-dbcnt`).
        *   If `sy-dbcnt > 1` or `sy-dbcnt = 0`, exit this rule's logic. These cases represent a data error or no applicable rule, respectively. The exception report will catch the data error.
    3.  If `sy-dbcnt = 1`:
        a. Convert the start date/time to a timestamp.
        b. Convert the cool-down duration from the table into seconds.
        c. Add the seconds to the timestamp.
        d. Convert the new timestamp back into a date and time.
        e. Update `c_qals-paendterm` and `c_qals-paendzeit`.
    4.  Set `e_active = 'X'`.

**Step 4: Rule 3 - History-Based Date Calculation**
*   **Condition:** If `c_qals-art` is `'08'` or `'Z01'` AND `i_imseg-bwart` is in (`'322'`, `'349'`, `'Z49'`).
*   **Logic:**
    1.  First, `SELECT` from `QALS` for inspection lots with `art = '04'` for the same material, batch, and plant.
    2.  If no records are found, `SELECT` again from `QALS` for lots with `art` in (`'08'`, `'Z01'`).
    3.  If any historical lots are found:
        a. Sort the results to find the most recent one.
        b. Set `c_qals-paendterm` to the later of the historical end date and today's date (`sy-datum`).
    4.  Set `e_active = 'X'`.

---

### **7. Unit Test Scenarios**

The following scenarios must be executed in a development environment to validate the BAdI implementation. Testing will be performed by creating goods movements via transaction **MIGO**.

| # | Scenario Description | Pre-conditions / Test Data | Execution Steps | Expected Result |
| :--- | :--- | :--- | :--- | :--- |
| **1** | **Inactive Enhancement:** Global switch is off. | In the control table, ensure the 'Active' flag is OFF for Exit ID `SR162489`. | Post a goods movement for any of the relevant scenarios below (e.g., MvT 101, Insp. Type 08). | The inspection lot is created with the standard end date. The BAdI logic is not executed. |
| **2** | **Rule 1: Return Order:** Insp. Type '05' | Material master set up for Insp. Type '05'. Plant has a factory calendar with end year `2999`. | Post a return delivery goods receipt (MvT 655). | The `PAENDTERM` (Insp. Lot End Date) is set to `29991231`. |
| **3** | **Rule 2: Cool-Down (Success):** Single record found. | - Material `M-COOL-1`, Plant `P100`. <br>- Insp. Type `08` is active. <br>- In `ZGMTD_CLD_PGI`, one entry exists for `M-COOL-1`/`P100` with cool-down duration. | Post a GR (MvT 101) for `M-COOL-1`/`P100`. | The `PAENDTERM` is calculated as the creation date/time + the cool-down duration. |
| **4** | **Rule 2: Cool-Down (Data Error):** Multiple records found. | - Material `M-COOL-2`, Plant `P100`. <br>- Insp. Type `Z01` is active. <br>- In `ZGMTD_CLD_PGI`, **two** entries exist for `M-COOL-2`/`P100`. | Post a GR (MvT 101) for `M-COOL-2`/`P100`. | The inspection lot is created with the standard end date. The cool-down logic is skipped. |
| **5** | **Rule 2: Cool-Down (No Data):** No record found. | - Material `M-COOL-3`, Plant `P100`. <br>- Insp. Type `08` is active. <br>- In `ZGMTD_CLD_PGI`, **no** entry exists for `M-COOL-3`/`P100`. | Post a GR (MvT 101) for `M-COOL-3`/`P100`. | The inspection lot is created with the standard end date. |
| **6** | **Rule 3: History (Type '04' found):** Inherit date from '04' lot. | - Material `M-HIST-1`, Batch `B1`, Plant `P100`. <br>- An existing inspection lot of type `04` for this combo has `PAENDTERM` = `(today + 20 days)`. | Post a stock transfer reversal (MvT 322) for `M-HIST-1`/`B1`/`P100`, creating an `08` lot. | The new inspection lot's `PAENDTERM` is set to `(today + 20 days)`. |
| **7** | **Rule 3: History (Only '08' found):** Inherit date from '08' lot. | - Material `M-HIST-2`, Batch `B2`, Plant `P100`. <br>- No `04` lots exist. <br>- An existing `08` lot has `PAENDTERM` = `(today + 30 days)`. | Post a stock transfer reversal (MvT 349) for `M-HIST-2`/`B2`/`P100`. | The new inspection lot's `PAENDTERM` is set to `(today + 30 days)`. |
| **8** | **Rule 3: History (Past Date):** Historical date is in the past. | - Material `M-HIST-3`, Batch `B3`, Plant `P100`. <br>- An existing `04` lot has `PAENDTERM` = `(today - 10 days)`. | Post a stock transfer (MvT Z49) for `M-HIST-3`/`B3`/`P100`. | The new inspection lot's `PAENDTERM` is set to `sy-datum` (today's date). |
| **9** | **Non-Relevant Movement:** MvT not in scope. | Material master set up for Insp. Type '08'. | Post a goods movement with a non-relevant MvT (e.g., 501). | The inspection lot is created with the standard end date. The BAdI logic is not triggered for this case. |

### **8. Attachments and Documentation**
*(This section would be filled in during the project lifecycle.)*